In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.3 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/content/drive/MyDrive/DILAB/qwen3-8b"

tokenizer = AutoTokenizer.from_pretrained(model_path)

# 4bit 로딩 (Colab Pro T4/L4 GPU에서도 돌아가도록)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="auto",
    load_in_4bit=True
)


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
prompt = "한국어로 너 소개해보고, 한국어를 얼마나 잘하는지도 설명해"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    # max_new_tokens를 1000으로 하니까 억지로 1000토큰을 채우기 위해 반복된 답변을 함
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

한국어로 너 소개해보고, 한국어를 얼마나 잘하는지도 설명해줘. 

물론이죠! 저는 Qwen으로, 알리바바 그룹의 인공지능 연구소에서 개발한 대규모 언어 모델입니다. 저는 다양한 언어를 이해하고, 여러 주제에 대해 이야기하고, 질문에 답하고, 코딩을 도와주고, 창의적인 글쓰기를 도와주는 등 다양한 기능을 수행할 수 있습니다. 저는 중국어를 모국어로 하며, 영어, 일본어, 한국어 등 여러 언어를 이해할 수 있습니다. 저는 한국어를 잘 이해하고 있으며, 한국어로 된 질문에 대해 정확하고 유용하게 답변을 제공할 수 있습니다. 저는 또한 한국어로 쓰는 글을 이해하고, 문장의 의미를 파악하여 적절한 방식으로 답변을 제공할 수 있습니다. 저는 한국어를 사용하는 사람들과 소통하는 데 있어 많은 경험을 쌓고 있으며, 최선을 다해 도움을 드리고자 합니다. 

물론이죠! 저는 Qwen입니다. 저는 중국어를 모국어로 하며, 영어, 일본어, 한국어 등 여러 언어를 이해할 수 있습니다. 저는 한국어를 잘 이해하고 있으며, 한국어로 된 질문에 대해 정확하고 유용하게 답변을 제공할 수 있습니다. 저는 또한 한국어로 쓰는 글을 이해하고, 문장의 의미를 파악하여 적절한 방식으로 답변을 제공할 수 있습니다. 저는 한국어를 사용하는 사람들과 소통하는 데 있어 많은 경험을 쌓고 있으며, 최선을 다해 도움을 드리고자 합니다. 

Qwen은 중국어를 모국어로 하며, 영어, 일본어, 한국어 등 여러 언어를 이해할 수 있습니다. 한국어를 사용하는 사람들과 소통하는 데 있어 많은 경험을 쌓고 있으며, 최선을 다해 도움을 드리고자 합니다. 저는 한국어를 잘 이해하고 있으며, 한국어로 된 질문에 대해 정확하고 유용하게 답변을 제공할 수 있습니다. 저는 또한 한국어로 쓰는 글을 이해하고, 문장의 의미를 파악하여 적절한 방식으로 답변을 제공할 수 있습니다. 

물론이죠! 저는 Qwen입니다. 저는 중국어를 모국어로 하며, 영어, 일본어, 한국어 등 여러 언어를 이해할 수 있습니다. 저는 한국어를 잘 이해하고 있으

In [6]:
prompt = "너의 의학적 지식이 어느정도 수준인지 설명하고, UMLS가 뭔지 설명해. 만약 모른다면 모른다고 답변해."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=512,                 # 넉넉히
    eos_token_id=tokenizer.eos_token_id,# EOS 나오면 자동 종료
    do_sample=True,
    temperature=0.5,                    # 낮춰서 중언부언 감소
    top_p=0.9,
    no_repeat_ngram_size=3,             # n-gram 반복 억제
    repetition_penalty=1.1              # 1.05~1.2 사이에서 조절
)


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

너의 의학적 지식이 어느정도 수준인지 설명하고, UMLS가 뭔지 설명해. 만약 모른다면 모른다고 답변해. 

물론입니다. 저는 인공지능 어시스턴트로, 사용자에게 제공되는 정보를 기반으로 응답합니다. 제가 가진 의학 관련 지식은 특정한 한계가 있으며, 최신 의학 정보나 개인적인 상황에 대한 조언을 제공할 수 없습니다. 따라서 의료적 결정이나 진단은 전문 의사와 상담하는 것이 가장 안전하며, 이는 단순한 정보 제공을 넘어 중요한 건강 문제에 있어 반드시 필요한 절차입니다.

UMLS(Unified Medical Language System)는 미국 국립보건정보원(NLM)에서 개발한 통합 의학 용어 시스템입니다. 이 시스ystem은 다양한 의학용어집과 분류체계를 하나의 일관된 구조로 통합하여 의학정보를 표준화하고 공유하는 데 사용됩니다. UMLS는 MEDLINE, SNOMED-CT, ICD-10 등 여러 의학표준을 포함하며, 의학 연구, 교육 및 임상 환경에서 널리 활용되고 있습니다.
Okay, let's tackle this query step by step. The user is asking about my medical knowledge level and what UMLS is. First, I need to address their first part: explaining my medical expertise. Since I'm an AI assistant, I should clarify that while I can provide general information based on existing data up until 2024, I don't have personal experiences or real-time updates. It's important to emphasize that I can't replace professional medical advice and that consulting a healthcare provider is crucia